In [1]:
from xtquant import xtdata
import time
import threading

In [2]:
# 设定一个标的列表
code_list = ["000001.SZ", "000058.SZ", "000016.SZ", "300368.SZ", "600601.SH", "002091.SZ"]
# 设定获取数据的周期
period = "1d"

print("hello world")

hello world


In [ ]:
# 下载标的行情数据
if 1:
    print("code_list: ", code_list)
    for i in code_list:
        print("Downloading data for code: ", i)
        xtdata.download_history_data(i, period=period, incrementally=True, start_time='20240101', end_time='')  # 增量下载行情数据（开高低收,等等）到本地
        time.sleep(1)  # 间隔1秒，防止下载过快被服务器封IP

    

print("=" * 20)

code_list:  ['000001.SZ', '000058.SZ', '000016.SZ', '300368.SZ', '600601.SH', '002091.SZ']


In [8]:
# 读取本地历史行情数据
history_data = xtdata.get_market_data_ex([],code_list,period=period, start_time='20250101', end_time='', count=10, dividend_type='front', fill_data=True)  # 读取本地历史行情数据
print(type(history_data))
print(history_data.keys())
print(history_data[code_list[0]].keys())
for i in code_list:
    print(i)
    close_prices = [f"{price:.2f}" for price in history_data[i]["close"]]
    print(close_prices)

print("=" * 20)

<class 'dict'>
dict_keys(['000001.SZ', '000058.SZ', '000016.SZ', '300368.SZ', '600601.SH', '002091.SZ'])
Index(['time', 'open', 'high', 'low', 'close', 'volume', 'amount',
       'settelementPrice', 'openInterest', 'preClose', 'suspendFlag'],
      dtype='object')
000001.SZ
['11.51', '11.66', '11.63', '11.67', '11.59', '11.61', '11.85', '11.84', '11.97', '11.51']
000058.SZ
['9.17', '9.37', '9.44', '8.95', '9.12', '9.16', '9.24', '8.87', '8.97', '8.93']
000016.SZ
['5.80', '5.50', '5.55', '5.37', '5.42', '5.42', '5.48', '5.26', '5.30', '5.34']
300368.SZ
['4.68', '4.77', '4.79', '4.72', '4.68', '4.75', '4.85', '4.91', '5.13', '5.10']
600601.SH
['4.73', '4.77', '4.87', '4.91', '4.90', '5.05', '4.99', '4.91', '5.40', '5.43']
002091.SZ
['7.74', '7.69', '7.72', '7.72', '7.89', '8.00', '7.90', '7.97', '7.95', '7.88']


In [ ]:
# 向服务器订阅数据
for i in code_list:
    xtdata.subscribe_quote(i,period=period,count=-1) # 设置count = -1来取到当天所有实时行情

# 等待订阅完成
time.sleep(1)

In [ ]:
# 获取订阅后的行情
kline_data = xtdata.get_market_data_ex([],code_list,period=period, start_time='20240101', end_time='', count=10, dividend_type='front', fill_data=True)
print(type(kline_data))
print(kline_data.keys())
print(kline_data[code_list[0]].keys())
for i in code_list:
    print(i)
    close_prices = [f"{price:.2f}" for price in kline_data[i]["close"]]
    print(close_prices)

In [ ]:
# 获取订阅后的行情，并以固定间隔进行刷新,预期会循环打印10次
for i in range(10):
    # 这边做演示，就用for来循环了，实际使用中可以用while True
    kline_data = xtdata.get_market_data_ex([],code_list,period=period, start_time='20240101', end_time='', count=1, dividend_type='front', fill_data=True)
    for i in code_list:
        print(i)
        close_prices = [f"{price:.2f}" for price in kline_data[i]["close"]]
        print(close_prices)
    # print(kline_data)
    time.sleep(3) # 三秒后再次获取行情
    print("=" * 20)

In [ ]:
# 如果不想用固定间隔触发，可以以用订阅后的回调来执行
# 这种模式下当订阅的callback回调函数将会异步的执行，每当订阅的标的tick发生变化更新，callback回调函数就会被调用一次
# 本地已有的数据不会触发callback

# 定义的回测函数
## 回调函数中，data是本次触发回调的数据，只有一条
def f(data):
    print("回调函数被触发")
    print("回调数据:",data)
    
    code_list = list(data.keys())    # 获取到本次触发的标的代码

    kline_in_callback = xtdata.get_market_data_ex([],code_list,period = period, start_time='20250101', end_time='', count=10, dividend_type='none', fill_data=True)    # 在回调中获取klines数据
    # print("获取的行情数据:", kline_in_callback)
    for i in code_list:
        print(i)
        close_prices = [f"{price:.2f}" for price in kline_in_callback[i]["close"]]
        print(close_prices)
        time.sleep(3)
    print("=" * 20)

# 手动触发回调函数进行测试
# test_data = {'000001.SZ': {'time': '2025-03-17 09:30:00',  'price': 10.0}}
# f(test_data)

for i in code_list:
    xtdata.subscribe_quote(i,period=period, start_time='20250301', end_time='' ,count=10,callback=f) # 订阅时设定回调函数

# 使用回调时，必须要同时使用xtdata.run()来阻塞程序，否则程序运行到最后一行就直接结束退出了。
# xtdata.run()